# Packages

In [4]:
import cv2                                # State-of-the-art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import mediapipe as mp                    # pose estimation through mp
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API

# Setup

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Stream Feed

## Stream Setup

In [6]:
# Initialize to retrieve the camera flow
pipe = rs.pipeline()
cfg = rs.config()

# Define the format of both stream
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Align both cameras
align = rs.align(rs.stream.color)

## Stream Test

In [8]:
# Start the capture
pipe.start(cfg)

while True:
  # Wait for a coherent pair of frames "depth and color" and align them
  frame = pipe.wait_for_frames()
  aligned_frame = align.process(frame)
  
  # Retrieve the depth and color flow
  depth_frame = frame.get_depth_frame()
  color_frame = frame.get_color_frame()
  
  # Apply filters to reduce the treatment charge by decreasing the resolution
  decimation = rs.decimation_filter()
  decimation.set_option(rs.option.filter_magnitude, 1)
  
  # Apply smothering filter
  spatial = rs.spatial_filter()
  spatial.set_option(rs.option.filter_magnitude, 5)
  spatial.set_option(rs.option.filter_smooth_alpha, 1)
  spatial.set_option(rs.option.filter_smooth_delta, 50)
  
  # Apply filters to fill the holes
  hole_filling = rs.hole_filling_filter()
  
  # Retrieve the images from both flow (colorized depth)
  colorizer = rs.colorizer()
  color_image = np.asanyarray(color_frame.get_data())
  decimated_depth = decimation.process(depth_frame)
  smoothed_depth = spatial.process(decimated_depth)
  filled_depth = hole_filling.process(smoothed_depth)
  colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
  
  cv2.imshow("Depth", colorized_depth_frame)
  cv2.imshow("Color", color_image)
  
  if cv2.waitKey(1) == ord('q'):
    break
      
pipe.stop()
cv2.destroyAllWindows()

# MediaPipe Application

In [14]:
# Setup mediapipe instance
with mp_pose.Pose() as pose:
    # Start the capture
    pipe.start(cfg)
    
    while True:
        # Wait for a coherent pair of frames "depth and color" and align them
        frame = pipe.wait_for_frames()
        aligned_frame = align.process(frame)
        
        # Retrieve the depth and color flow
        depth_frame = frame.get_depth_frame()
        color_frame = frame.get_color_frame()
        
        # Apply filters to reduce the treatment charge by decreasing the resolution
        decimation = rs.decimation_filter()
        decimation.set_option(rs.option.filter_magnitude, 1)
        
        # Apply smothering filter
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.filter_magnitude, 5)
        spatial.set_option(rs.option.filter_smooth_alpha, 1)
        spatial.set_option(rs.option.filter_smooth_delta, 50)
        
        # Apply filters to fill the holes
        hole_filling = rs.hole_filling_filter()
        
        # Retrieve the images from both flow (colorized depth)
        colorizer = rs.colorizer()
        color_image = np.asanyarray(color_frame.get_data())
        decimated_depth = decimation.process(depth_frame)
        smoothed_depth = spatial.process(decimated_depth)
        filled_depth = hole_filling.process(smoothed_depth)
        colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
        
        # Convert color image to RGB
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        
        # Add MP on the colored image
        results = pose.process(color_image_rgb)
        
        # Render detections
        mp_drawing.draw_landmarks(color_image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )
        
        # Convert RGB image back to BGR for displaying
        color_image_bgr = cv2.cvtColor(color_image_rgb, cv2.COLOR_RGB2BGR)

        cv2.imshow("Depth", colorized_depth_frame)
        cv2.imshow("Color", color_image)
        cv2.imshow("MediaPipe", color_image_bgr)
        
        if cv2.waitKey(1) == ord('q'):
            break
            
    pipe.stop()
    cv2.destroyAllWindows()


# MediaPipe Application and Depth 

In [16]:
# Setup mediapipe instance
with mp_pose.Pose() as pose:
    # Start the capture
    pipe.start(cfg)
    
    while True:
        # Wait for a coherent pair of frames "depth and color" and align them
        frame = pipe.wait_for_frames()
        aligned_frame = align.process(frame)
        
        # Retrieve the depth and color flow
        depth_frame = frame.get_depth_frame()
        color_frame = frame.get_color_frame()
        
        # Apply filters to reduce the treatment charge by decreasing the resolution
        decimation = rs.decimation_filter()
        decimation.set_option(rs.option.filter_magnitude, 1)
        
        # Apply smothering filter
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.filter_magnitude, 5)
        spatial.set_option(rs.option.filter_smooth_alpha, 1)
        spatial.set_option(rs.option.filter_smooth_delta, 50)
        
        # Apply filters to fill the holes
        hole_filling = rs.hole_filling_filter()
        
        # Retrieve the images from both flow (colorized depth)
        colorizer = rs.colorizer()
        color_image = np.asanyarray(color_frame.get_data())
        
        decimated_depth = decimation.process(depth_frame)
        smoothed_depth = spatial.process(decimated_depth)
        filled_depth = hole_filling.process(smoothed_depth)
        depth_image = np.asanyarray(filled_depth.get_data())
        colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
        
        # Convert color image to RGB
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        
        # Add MP on the colored image
        results = pose.process(color_image_rgb)
        
        # Render detections
        mp_drawing.draw_landmarks(color_image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )
        
        # Convert RGB image back to BGR for displaying
        color_image_bgr = cv2.cvtColor(color_image_rgb, cv2.COLOR_RGB2BGR)
        
        # Retrieve and print the coordinates for each landmark
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * color_image.shape[1]) 
                y = int(landmark.y * color_image.shape[0])  
                
                # Print the calculated coordinates
                print(f"X = {x}, Y = {y}")
                
                # Ensure that the calculated pixel coordinates are within the bounds of the depth frame
                if 0 <= y < color_image.shape[0] and 0 <= x < color_image.shape[1]:
                    depth_value = depth_image[y, x]
                    print(f"Z = {depth_value}")
                else:
                    print("Coordinates out of bounds")

        cv2.imshow("Depth", colorized_depth_frame)
        cv2.imshow("Color", color_image)
        cv2.imshow("MediaPipe", color_image_bgr)
        
        if cv2.waitKey(1) == ord('q'):
            break
            
    pipe.stop()
    cv2.destroyAllWindows()

X = 371, Y = 119
Z = 671
X = 382, Y = 106
Z = 665
X = 386, Y = 108
Z = 663
X = 391, Y = 111
Z = 660
X = 366, Y = 100
Z = 677
X = 358, Y = 97
Z = 664
X = 351, Y = 94
Z = 4025
X = 389, Y = 121
Z = 657
X = 327, Y = 98
Z = 4025
X = 373, Y = 142
Z = 671
X = 350, Y = 132
Z = 666
X = 372, Y = 207
Z = 722
X = 235, Y = 149
Z = 1040
X = 388, Y = 319
Z = 700
X = 187, Y = 274
Z = 911
X = 442, Y = 230
Z = 836
X = 312, Y = 186
Z = 586
X = 460, Y = 217
Z = 624
X = 346, Y = 175
Z = 656
X = 462, Y = 208
Z = 603
X = 312, Y = 145
Z = 4470
X = 441, Y = 201
Z = 732
X = 312, Y = 151
Z = 4470
X = 292, Y = 410
Z = 575
X = 188, Y = 403
Z = 469
X = 463, Y = 363
Z = 846
X = 298, Y = 349
Z = 432
X = 335, Y = 534
Coordinates out of bounds
X = 260, Y = 558
Coordinates out of bounds
X = 300, Y = 549
Coordinates out of bounds
X = 237, Y = 573
Coordinates out of bounds
X = 349, Y = 624
Coordinates out of bounds
X = 292, Y = 666
Coordinates out of bounds
X = 383, Y = 122
Z = 669
X = 389, Y = 111
Z = 670
X = 392, Y = 11